In [2]:
import duckdb

duckdb.sql("""
CREATE OR REPLACE VIEW commune AS
FROM 'https://www.insee.fr/fr/statistiques/fichier/7766585/v_commune_2024.csv';
""")

In [4]:
import pandas as pd

duckdb.sql("""
SELECT *
  FROM commune limit 3
""").to_df()

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None
1,COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None
2,COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None


In [ ]:

    Quel est le code région de Mayotte ?
    Combien de communes y a-t-il en Guyane ?

In [3]:
duckdb.sql("""
SELECT dep, reg
  FROM commune
 WHERE NCC = 'MAMOUDZOU';
""").to_df()


,DEP,REG
0,976,6


In [5]:
duckdb.sql("""
SELECT COUNT(*)
  FROM commune
 WHERE dep = '973';
""").to_df()

,count_star()
0,22


In [8]:
duckdb.sql("""
SELECT COUNT(1)
  FROM commune
 WHERE dep IN ('02','60','80')
   AND NCC LIKE '%L%L%L%L%';
""").to_df()

,count(1)
0,32


    Affichez le nombre de communes par département
        Classez par nombre de communes décroissant
        Quel est le numéro du département se classant au 20e rang ?
    Quel est le numéro du département dans lequel se trouve le nom de commune le plus long


In [10]:
duckdb.sql("""
select dep, count(distinct com) as nb_communes from commune group by dep order by nb_communes desc limit 5;""")

┌─────────┬─────────────┐
│   DEP   │ nb_communes │
│ varchar │    int64    │
├─────────┼─────────────┤
│ NULL    │        2564 │
│ 62      │         890 │
│ 02      │         798 │
│ 80      │         772 │
│ 57      │         725 │
└─────────┴─────────────┘

In [7]:
duckdb.sql("""select * from commune where dep is null;""")

┌─────────┬─────────┬───────┬─────────┬─────────┬─────────┬───────┬──────────────────────────┬──────────────────────────┬──────────────────────────┬─────────┬───────────┐
│ TYPECOM │   COM   │  REG  │   DEP   │  CTCD   │   ARR   │ TNCC  │           NCC            │          NCCENR          │         LIBELLE          │   CAN   │ COMPARENT │
│ varchar │ varchar │ int64 │ varchar │ varchar │ varchar │ int64 │         varchar          │         varchar          │         varchar          │ varchar │  varchar  │
├─────────┼─────────┼───────┼─────────┼─────────┼─────────┼───────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼─────────┼───────────┤
│ COMD    │ 01015   │  NULL │ NULL    │ NULL    │ NULL    │     1 │ ARBIGNIEU                │ Arbignieu                │ Arbignieu                │ NULL    │ 01015     │
│ COMD    │ 01025   │  NULL │ NULL    │ NULL    │ NULL    │     0 │ BAGE LA VILLE            │ Bâgé-la-Ville            │ Bâgé-la-Ville          

duckdb.sql("""
SELECT dep,
       COUNT(1) AS nb_communes,
       RANK() OVER (ORDER BY COUNT(1) DESC) AS rang
  FROM commune
 WHERE dep IS NOT NULL
 GROUP BY dep
HAVING nb_communes >= 500
 ORDER BY 2 DESC;
""").to_df()

In [ ]:
duckdb.sql("""
SELECT dep,
       COUNT(1) AS nb_communes,
       RANK() OVER (ORDER BY COUNT(1) DESC) AS rang
  FROM commune
 WHERE dep IS NOT NULL
 GROUP BY dep
HAVING nb_communes >= 500
 ORDER BY rang DESC;
""").to_df()

,DEP,nb_communes,rang
0,62,890,1
1,02,798,2
2,80,772,3
3,57,725,4
4,76,708,5
5,21,698,6
6,60,680,7
7,59,648,8
8,51,611,9
9,54,591,10


    Quel est le numéro du département dans lequel se trouve le nom de commune le plus long


In [6]:
duckdb.sql("""SELECT max(LENGTH(ncc))
                        FROM commune""")

┌──────────────────┐
│ max(length(ncc)) │
│      int64       │
├──────────────────┤
│               45 │
└──────────────────┘

# test

In [ ]:
duckdb.sql("""
SELECT dep
  FROM commune
 WHERE LENGTH(ncc) = (SELECT max(LENGTH(ncc))
                        FROM commune);
""").to_df()

,DEP
0,51
